# NBA and WNBA Short Charts using NBA API and Plotly 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import requests
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import plotly.graph_objects as go
import plotly.express as px
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

fig_DIR = "../figs/hex_maps/"

## Chose Player data to plot

In [ ]:
year = 2023
season_type = "Regular Season"
player_name = 'Stephen Curry'
league, league_id, season = 'NBA','00',str(year)+'-'+str(year+1)[-2:]
# player_name = 'Candace Parker'
# league, league_id, season = 'WNBA','10',str(year)
# player_name = 'Jonathan Kuminga'
# league, league_id, season = 'GLeague','20',str(year)
player_dict = get_players(league=league,from_year=year, to_year=year)
player_id = [player['pID'] for player in player_dict if player['Name'] == player_name][0]
player_shotchart = ShotChartDetail(league_id=league_id, team_id=0, player_id=player_id, context_measure_simple='FGA')#, season_nullable=season)
if league == 'GLeague' and year==2020:
    season = str(year+1)

shots_df = player_shotchart.get_data_frames()[0]
shots_df['LOC_X']=-shots_df['LOC_X']
league_avg = player_shotchart.get_data_frames()[1]
binned_df = create_bins(data_frame=shots_df, league_average=league_avg)
data_frame = binned_df
dropped_dups = data_frame.drop_duplicates(subset=['BIN_LOC_X', 'BIN_LOC_Y'], keep='first')
dropped_dups = dropped_dups.loc[dropped_dups.BIN_LOC_Y < 417.5]
if league=='NBA':
    dropped_dups = dropped_dups.loc[dropped_dups.LOC_RAW_COUNTS > 1]
# dropped_dups.head()

## Figure Settings and Plot 

In [ ]:
bball_gray = '#312f30'
bball_white = '#dddee0'
bball_orange = '#f87c24'
bball_light_orange = '#fbaf7b'
bball_black = '#000010'
dark_grey = '#282828'

colorscale = 'Plasma'

fig = go.Figure()

draw_plotly_court(fig,lw=1,margins=13,lcolor=bball_orange)
fig.add_trace(go.Scatter(
    x=dropped_dups.BIN_LOC_X, y=dropped_dups.BIN_LOC_Y, mode='markers', name='markers',
    text = dropped_dups.LOC_ZONE_PERCENTAGE	,
    marker=dict(
        size=dropped_dups.LOC_COUNTS, sizemode='area', sizeref=1.5, 
        sizemin=2.5,
        color=dropped_dups.PCT_LEAGUE_COMPARISON_ZONE, colorscale=colorscale,
        colorbar=dict(
            thickness=15,
            x=0.85,
            y=0.87,
            yanchor='middle',
            len=0.2,
            title=dict(
                text="<B>Accuracy</B>",
                font=dict(
                    size=11,
                    color='White'
                ),
            ),
            tickvals=[-10, 0, 10],
            ticktext=['Worse','Average','Better'],
            tickfont=dict(
                size=11,
                color='White'
            )
        ),
        cmin=-10, cmax=10,
        line=dict(width=0.5, color='White'), symbol='hexagon',
    ),
    hovertemplate ='<i>FG%</i>: %{text:.0f}<extra></extra>',
))
layout_update_plotly(fig,player_name,season, league, season_type, bball_black)
fig.write_image(fig_DIR+"Hex Map {0} {1}.png".format(player_name,season),scale=5)
fig.show()